In [ ]:
from google.colab import drive

# Attach drive
drive.mount("/content/drive")

In [ ]:
# Global var for folder to save results logs to
DRIVE_FOLDER_OUT = "/content/drive/MyDrive/Colab Files/wiki_entity_logs/"

In [ ]:
!git clone https://github.com/daniel-furman/Capstone.git
!pip install -r /content/Capstone/requirements.txt
#!pip install -r /content/Capstone/requirements_llama.txt

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import json
from json import JSONDecodeError
import urllib.parse
import urllib.request

from argparse import Namespace

import os

from datetime import date

import re
from ftfy import fix_text
from string import punctuation

import spacy

os.chdir("/content/Capstone/src/wikipedia_entity_analysis/"
from wiki_analysis import CODE_TO_LANG_DICT, CODE_TO_WIKI_CLEANUP_DICT, CODE_TO_SPACY_MODEL_DICT, load_spacy_models, get_mulitlingual_lookup, get_wikipedia_pages, get_article_info, count_entities_in_article, 


In [ ]:
entity_analysis_df = pd.read_csv(
    "../../data/error_analysis/entity_analysis_language_and_accuracy_by_entity.csv"
)

In [ ]:
# args config for grabbing entities
args = Namespace(
    language="en",
    iterations=100,
    cleanup_str=CODE_TO_WIKI_CLEANUP_DICT["en"],
    debug=False
)

In [ ]:
# get entity <-> multilingual translation lookup resolver
target_entities_multiling = get_mulitlingual_lookup(get_mulitlingual_lookup)

In [ ]:
# load spacy model
model_name = CODE_TO_SPACY_MODEL_DICT[args.language]
!python -m spacy download {model_name}

In [ ]:
spacy_model = load_spacy_models(CODE_TO_SPACY_MODEL_DICT[args.language])

article_titles = []
article_ids = []
article_word_counts = []
article_full_entities = []
article_target_entities = []
article_summary_strs = []
for i in range(args.iterations):
    # get pages to parse
    pages_to_parse = get_wikipedia_pages(args.language, args.debug)

    # for each page
    for page in pages_to_parse:
        article_data = []
        article_id = list(page.keys())[0]
        article_title = list(page.items())[0]
        # get info
        article_info = get_article_info(article_title, article_id, args.language, cleanup_str, args.debug)
        # get stats
        article_word_count, article_full_entities, article_target_entities = count_entities_in_article(target_entities_multiling, article_info, spacy_model, args.language, debug)
        article_summary_str = f"article titled {article_title} with pageid {article_id} has {article_word_count} words, {len(article_full_entities)} unique entity mentions {sum(article_full_entities.values())} total entity mentions, {len(article_target_entities)} unique target entities and {sum(article_target_entities.values())} total target entity mentions"
        
        if args.debug:
            print(article_summary_str)
        
        # commit stats
        article_titles.append(article_title)
        article_ids.append(article_id)
        article_word_counts.append(article_word_count)
        article_full_entities.append(article_full_entities)
        article_target_entities.append(article_target_entities)
        article_summary_strs.append(article_summary_str)
        break
    break

log = pd.DataFrame({'article_title': article_titles, 'article_id': article_id, 'article_word_count': article_word_counts, 'article_full_entities': article_full_entites, 'article_target_entities': article_target_entities, 'article_summary': article_summary_strs})
log_name = args.language + '-' + args.iterations + 'wikipedia-entity-count' + str(date.today())
log_new_path = os.path.join(DRIVE_FOLDER_OUT, log_name)
log.to_csv(log_new_path, encoding='utf-8', index=False))

In [ ]:
# save result logs to drive

log = glob.glob(f"/content/Capstone/src/fact_completion_scripts/{log_fpath}")[0]
log_name = args.language + '-' + log.split("/")[-1]
log_new_path = os.path.join(DRIVE_FOLDER_OUT, log_name)
!cp {log} '{log_new_path}'